In [10]:
import sys 
sys.path.append("../")

In [11]:
import pandas as pd 
import datetime as dt
import plotly.graph_objects as go 
from technical.indicators import rsi 
from technical.patterns import apply_patterns
from plotting import CandlePlot

In [12]:
df_raw = pd.read_pickle("../data/EUR_USD_H1.pkl")

In [13]:
df_raw.shape

(59621, 14)

In [14]:
df_an = df_raw.copy()#df_raw.iloc[-6000:].copy()
df_an.reset_index(drop=True, inplace=True)

In [15]:
df_an.shape

(59621, 14)

In [16]:
df_an = rsi(df_an)
df_an.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,RSI_14
0,2015-06-01 00:00:00+00:00,1195,1.09573,1.09628,1.09346,1.09424,1.09565,1.09620,1.09337,1.09415,1.09581,1.09636,1.09355,1.09433,NaN
1,2015-06-01 01:00:00+00:00,922,1.09424,1.09473,1.09305,1.09420,1.09416,1.09465,1.09297,1.09412,1.09433,1.09481,1.09313,1.09427,NaN
2,2015-06-01 02:00:00+00:00,656,1.09418,1.09540,1.09404,1.09509,1.09409,1.09531,1.09395,1.09501,1.09426,1.09549,1.09412,1.09517,NaN
3,2015-06-01 03:00:00+00:00,443,1.09506,1.09556,1.09481,1.09502,1.09498,1.09549,1.09473,1.09495,1.09513,1.09564,1.09489,1.09509,NaN
4,2015-06-01 04:00:00+00:00,813,1.09498,1.09652,1.09491,1.09598,1.09491,1.09644,1.09484,1.09590,1.09505,1.09659,1.09498,1.09605,NaN


In [17]:
df_an = apply_patterns(df_an)

In [18]:
df_an["EMA_200"] = df_an.mid_c.ewm(span=200, min_periods=200).mean()

In [19]:
df_an.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'RSI_14',
       'body_lower', 'body_upper', 'body_bottom_perc', 'body_top_perc',
       'body_perc', 'direction', 'body_size', 'low_change', 'high_change',
       'body_size_change', 'mid_point', 'mid_point_prev_2', 'body_size_prev',
       'direction_prev', 'direction_prev_2', 'body_perc_prev',
       'body_perc_prev_2', 'HANGING_MAN', 'SHOOTING_STAR', 'SPINNING_TOP',
       'MARUBOZU', 'ENGULFING', 'TWEEZER_TOP', 'TWEEZER_BOTTOM',
       'MORNING_STAR', 'EVENING_STAR', 'EMA_200'],
      dtype='object')

In [20]:
our_cols= ["time",'mid_o', 'mid_h', 'mid_l', 'mid_c','bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', "ENGULFING", 'direction', 'EMA_200', "RSI_14"]

In [21]:
df_slim = df_an[our_cols].copy()
df_slim.dropna(inplace=True)
df_slim.reset_index(drop=True, inplace=True)

In [22]:
df_slim.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,ENGULFING,direction,EMA_200,RSI_14
0,2015-06-11 07:00:00+00:00,1.12986,1.12996,1.12504,1.12628,1.12978,1.12988,1.12493,1.12618,1.12993,1.13004,1.12513,1.12637,False,-1,1.122955,37.475134
1,2015-06-11 08:00:00+00:00,1.12625,1.12688,1.12410,1.12667,1.12617,1.12682,1.12400,1.12660,1.12633,1.12696,1.12420,1.12674,False,1,1.122998,39.097003
2,2015-06-11 09:00:00+00:00,1.12666,1.12690,1.12462,1.12653,1.12659,1.12683,1.12454,1.12645,1.12674,1.12698,1.12469,1.12661,False,-1,1.123038,38.708834
3,2015-06-11 10:00:00+00:00,1.12650,1.12841,1.12632,1.12667,1.12643,1.12833,1.12625,1.12660,1.12656,1.12850,1.12640,1.12674,True,1,1.123080,39.357232
4,2015-06-11 11:00:00+00:00,1.12670,1.12688,1.12326,1.12337,1.12662,1.12681,1.12319,1.12329,1.12677,1.12695,1.12334,1.12345,True,-1,1.123083,31.025534


In [23]:
BUY = 1 
SELL = -1 
NONE = 0
RSI_LIMIT = 50.0 

def apply_signal(row):
    if row.ENGULFING == True:
        if row.direction == BUY and row.mid_l > row.EMA_200:
            if row.RSI_14 > RSI_LIMIT:
                return BUY 
        if row.direction == SELL and row.mid_h < row.EMA_200:
            if row.RSI_14 < RSI_LIMIT:
                return SELL
    return NONE
    

In [24]:
df_slim["SIGNAL"] = df_slim.apply(apply_signal, axis=1)

In [25]:
df_slim["SIGNAL"].value_counts()

SIGNAL
 0    54181
-1     2692
 1     2549
Name: count, dtype: int64

In [26]:
LOSS_FACTOR = -1.0 
PROFIT_FACTOR = 1.5 

def apply_take_profit(row):
    if row.SIGNAL != NONE: 
        if row.SIGNAL == BUY:
            return(row.ask_c - row.ask_o) * PROFIT_FACTOR + row.ask_c
        else: 
            return(row.bid_c - row.bid_o) * PROFIT_FACTOR + row.bid_c
    else: 
        return 0.0 
    
def apply_stop_loss(row):
    if row.SIGNAL != NONE:
        if row.SIGNAL == BUY:
            return row.ask_o
        else: 
            return row.bid_o
    else: 
        return 0.0

In [27]:
df_slim["TP"] = df_slim.apply(apply_take_profit, axis=1)
df_slim["SL"] = df_slim.apply(apply_stop_loss, axis=1)

In [28]:
df_slim[df_slim.SIGNAL==BUY].head()

,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL
37,2015-06-12 20:00:00+00:00,1.12539,1.12704,1.12519,1.12681,1.12531,1.12671,1.12508,1.12623,1.12547,1.12750,1.12529,1.12739,True,1,1.123392,56.166680,1,1.130270,1.12547
68,2015-06-16 03:00:00+00:00,1.12721,1.12799,1.12692,1.12794,1.12712,1.12791,1.12684,1.12787,1.12730,1.12808,1.12699,1.12802,True,1,1.123751,59.218660,1,1.129100,1.12730
71,2015-06-16 06:00:00+00:00,1.12800,1.13188,1.12593,1.13155,1.12789,1.13179,1.12584,1.13146,1.12810,1.13197,1.12602,1.13164,True,1,1.123925,69.635974,1,1.136950,1.12810
92,2015-06-17 03:00:00+00:00,1.12534,1.12672,1.12519,1.12618,1.12527,1.12663,1.12513,1.12611,1.12542,1.12681,1.12525,1.12625,True,1,1.124084,55.554384,1,1.127495,1.12542
100,2015-06-17 11:00:00+00:00,1.12528,1.12697,1.12470,1.12688,1.12519,1.12690,1.12463,1.12681,1.12536,1.12704,1.12478,1.12696,True,1,1.124244,56.011451,1,1.129360,1.12536


In [29]:
df_plot = df_slim.iloc[0:100]
cp = CandlePlot(df_plot, candles=True)

trades = cp.df_plot[cp.df_plot.SIGNAL != NONE]
markers = ["mid_c", "TP", "SL"]
marker_colors = ["#0000FF", "#00FF00", "#FF0000"]

for i in range(3):
    cp.fig.add_trace(go.Scatter(
        x = trades.sTime,
        y = trades[markers[i]],
        mode = "markers",
        marker=dict(color=marker_colors[i],size=12)
    ))

cp.show_plot(line_traces=["EMA_200"], sec_traces=["RSI_14"], height=250)

In [30]:
class Trade:
    def __init__(self, row):
        self.running = True
        self.start_index = row.name

        if row.SIGNAL == BUY:
            self.start_price = row.ask_c
            self.end_price = row.ask_c 
        else:
            self.start_price = row.bid_c
            self.end_price = row.bid_c   

        self.SIGNAL = row.SIGNAL 
        self.TP = row.TP 
        self.SL = row.SL 
        self.result = 0.0 
        self.end_time = row.time
        self.start_time = row.time 
        self.duration = 0 

    def close_trade(self, row, result, trigger_price):
        self.running = False 
        self.result = result 
        self.end_time = row.time
        self.trigger_price = trigger_price


    def update(self, row):
        self.duration += 1 
        if self.SIGNAL == BUY:
            if row.bid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.bid_h)
            elif row.bid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.bid_l)
             
        if self.SIGNAL == SELL:
            if row.ask_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.ask_l)
            elif row.ask_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.ask_h)
             


In [31]:
open_trades = []
closed_trades = []

for index, row in df_slim.iterrows():
    for ot in open_trades:
        ot.update(row)
        if ot.running == False:
            closed_trades.append(ot)
    open_trades = [x for x in open_trades if x.running == True]

    if row.SIGNAL != NONE:
       open_trades.append(Trade(row))


In [32]:
vars(closed_trades[0])
df_result = pd.DataFrame.from_dict([vars(x) for x in closed_trades])
df_result.result.sum()

np.float64(-765.0)

In [33]:
df_result.sort_values(by="start_index", inplace=True)

In [34]:
df_m5 = pd.read_pickle("../data/EUR_USD_M5.pkl")

In [35]:
df_m5.shape

(713626, 14)

In [36]:
from dateutil import parser 

In [37]:
df_m5_slim = df_m5[["time", "bid_h", "bid_l", "ask_h", "ask_l"]].copy()

In [38]:
df_m5_slim.head()

,time,bid_h,bid_l,ask_h,ask_l
0,2015-06-01 00:00:00+00:00,1.09607,1.09548,1.09623,1.09564
1,2015-06-01 00:05:00+00:00,1.09620,1.09575,1.09636,1.09592
2,2015-06-01 00:10:00+00:00,1.09593,1.09546,1.09607,1.09563
3,2015-06-01 00:15:00+00:00,1.09574,1.09495,1.09593,1.09511
4,2015-06-01 00:20:00+00:00,1.09501,1.09367,1.09518,1.09384


In [39]:
df_signals = df_slim[df_slim.SIGNAL != NONE].copy()

In [40]:
df_signals["m5_start"] = [x + dt.timedelta(hours=1) for x in df_signals.time]

In [41]:
df_signals["start_index_h1"] = df_signals.index

In [42]:
df_signals.head()

,time,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,...,ask_c,ENGULFING,direction,EMA_200,RSI_14,SIGNAL,TP,SL,m5_start,start_index_h1
25,2015-06-12 08:00:00+00:00,1.12200,1.12296,1.11512,1.11677,1.12194,1.12289,1.11503,1.11660,1.12206,...,1.11694,True,-1,1.123267,26.742318,-1,1.10859,1.12194,2015-06-12 09:00:00+00:00,25
37,2015-06-12 20:00:00+00:00,1.12539,1.12704,1.12519,1.12681,1.12531,1.12671,1.12508,1.12623,1.12547,...,1.12739,True,1,1.123392,56.166680,1,1.13027,1.12547,2015-06-12 21:00:00+00:00,37
52,2015-06-15 11:00:00+00:00,1.12170,1.12260,1.12042,1.12112,1.12162,1.12253,1.12034,1.12104,1.12178,...,1.12120,True,-1,1.123154,44.762202,-1,1.12017,1.12162,2015-06-15 12:00:00+00:00,52
68,2015-06-16 03:00:00+00:00,1.12721,1.12799,1.12692,1.12794,1.12712,1.12791,1.12684,1.12787,1.12730,...,1.12802,True,1,1.123751,59.218660,1,1.12910,1.12730,2015-06-16 04:00:00+00:00,68
71,2015-06-16 06:00:00+00:00,1.12800,1.13188,1.12593,1.13155,1.12789,1.13179,1.12584,1.13146,1.12810,...,1.13164,True,1,1.123925,69.635974,1,1.13695,1.12810,2015-06-16 07:00:00+00:00,71


In [43]:
df_signals.columns

Index(['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h', 'bid_l',
       'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'ENGULFING', 'direction',
       'EMA_200', 'RSI_14', 'SIGNAL', 'TP', 'SL', 'm5_start',
       'start_index_h1'],
      dtype='object')

In [44]:
df_signals.drop(['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c','ask_h', 'bid_o', 'bid_h', 'bid_l', 'ask_o', 'ask_l','ENGULFING','EMA_200', 'RSI_14', "direction"], axis=1, inplace=True)

In [45]:
df_signals.head()

,bid_c,ask_c,SIGNAL,TP,SL,m5_start,start_index_h1
25,1.11660,1.11694,-1,1.10859,1.12194,2015-06-12 09:00:00+00:00,25
37,1.12623,1.12739,1,1.13027,1.12547,2015-06-12 21:00:00+00:00,37
52,1.12104,1.12120,-1,1.12017,1.12162,2015-06-15 12:00:00+00:00,52
68,1.12787,1.12802,1,1.12910,1.12730,2015-06-16 04:00:00+00:00,68
71,1.13146,1.13164,1,1.13695,1.12810,2015-06-16 07:00:00+00:00,71


In [46]:
df_signals.rename(columns={
    "bid_c" : "start_price_BUY",
    "ask_c" : "start_price_SELL",
    "m5_start": "time"
}, inplace=True)

In [47]:
df_signals.head()

,start_price_BUY,start_price_SELL,SIGNAL,TP,SL,time,start_index_h1
25,1.11660,1.11694,-1,1.10859,1.12194,2015-06-12 09:00:00+00:00,25
37,1.12623,1.12739,1,1.13027,1.12547,2015-06-12 21:00:00+00:00,37
52,1.12104,1.12120,-1,1.12017,1.12162,2015-06-15 12:00:00+00:00,52
68,1.12787,1.12802,1,1.12910,1.12730,2015-06-16 04:00:00+00:00,68
71,1.13146,1.13164,1,1.13695,1.12810,2015-06-16 07:00:00+00:00,71


In [48]:
df_m5_slim.head(2)

,time,bid_h,bid_l,ask_h,ask_l
0,2015-06-01 00:00:00+00:00,1.09607,1.09548,1.09623,1.09564
1,2015-06-01 00:05:00+00:00,1.09620,1.09575,1.09636,1.09592


In [49]:
merged = pd.merge(left=df_m5_slim, right=df_signals, on="time", how="left")

In [50]:
merged[merged.SIGNAL.isna()==False].head()
merged.fillna(0, inplace=True)

In [51]:
merged.SIGNAL = merged.SIGNAL.astype(int)
merged.start_index_h1 = merged.start_index_h1.astype(int)

In [52]:
merged.head()

,time,bid_h,bid_l,ask_h,ask_l,start_price_BUY,start_price_SELL,SIGNAL,TP,SL,start_index_h1
0,2015-06-01 00:00:00+00:00,1.09607,1.09548,1.09623,1.09564,0.0,0.0,0,0.0,0.0,0
1,2015-06-01 00:05:00+00:00,1.09620,1.09575,1.09636,1.09592,0.0,0.0,0,0.0,0.0,0
2,2015-06-01 00:10:00+00:00,1.09593,1.09546,1.09607,1.09563,0.0,0.0,0,0.0,0.0,0
3,2015-06-01 00:15:00+00:00,1.09574,1.09495,1.09593,1.09511,0.0,0.0,0,0.0,0.0,0
4,2015-06-01 00:20:00+00:00,1.09501,1.09367,1.09518,1.09384,0.0,0.0,0,0.0,0.0,0


In [53]:
class TradeM5:
    def __init__(self, row):
        self.running = True
        self.start_index_m5 = row.name
        self.start_index_h1= row.start_index_h1

        if row.SIGNAL==BUY:

            self.start_price = row.start_price_BUY
            self.trigger_price = row.start_price_BUY
        else:
            self.start_price = row.start_price_SELL
            self.trigger_price = row.start_price_SELL
        #self.end_price = row.mid_c 
        self.SIGNAL = row.SIGNAL 
        self.TP = row.TP 
        self.SL = row.SL 
        self.result = 0.0 
        self.end_time = row.time
        self.start_time = row.time 
        self.duration = 1 

    def close_trade(self, row, result, trigger_price):
        self.running = False 
        self.result = result 
        self.end_time = row.time
        self.trigger_price = trigger_price


    def update(self, row):
        self.duration += 1 
        if self.SIGNAL == BUY:
            if row.bid_h >= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.bid_h)
            elif row.bid_l <= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.bid_l)
             
        if self.SIGNAL == SELL:
            if row.ask_l <= self.TP:
                self.close_trade(row, PROFIT_FACTOR, row.ask_l)
            elif row.ask_h >= self.SL:
                self.close_trade(row, LOSS_FACTOR, row.ask_h)
             


In [54]:
open_trades_m5 = []
closed_trades_m5= []

for index, row in merged.iterrows():

    if row.SIGNAL != NONE:
       open_trades_m5.append(TradeM5(row))

    for ot in open_trades_m5:
        ot.update(row)
        if ot.running == False:
            closed_trades_m5.append(ot)
    open_trades_m5 = [x for x in open_trades_m5 if x.running == True]

In [55]:
df_res_m5 = pd.DataFrame.from_dict([vars(x) for x in closed_trades_m5])

In [56]:
df_res_m5.head()

,running,start_index_m5,start_index_h1,start_price,trigger_price,SIGNAL,TP,SL,result,end_time,start_time,duration
0,False,2698,25,1.11694,1.12236,-1,1.108590,1.12194,-1.0,2015-06-12 12:10:00+00:00,2015-06-12 09:00:00+00:00,40
1,False,3022,52,1.12120,1.12198,-1,1.120170,1.12162,-1.0,2015-06-15 12:05:00+00:00,2015-06-15 12:00:00+00:00,3
2,False,3214,68,1.12787,1.12728,1,1.129100,1.12730,-1.0,2015-06-16 05:10:00+00:00,2015-06-16 04:00:00+00:00,16
3,False,3250,71,1.13146,1.12772,1,1.136950,1.12810,-1.0,2015-06-16 07:40:00+00:00,2015-06-16 07:00:00+00:00,10
4,False,3502,92,1.12611,1.12538,1,1.127495,1.12542,-1.0,2015-06-17 05:10:00+00:00,2015-06-17 04:00:00+00:00,16


In [57]:
df_res_m5.result.sum()

np.float64(-1584.0)

In [58]:
# with the spread attach to the trades 
